# Data Analysis - Module 6
## Real-World Capstone Projects

**Your Role:** Senior Data Analyst at TechFlow

**Your Mission:** Apply everything you've learned to solve real business problems.

**4 Projects:**
| # | Project | Departments |
|---|---------|-------------|
| 1 | Customer 360 Analysis | All Teams |
| 2 | Revenue Intelligence | Finance, Sales |
| 3 | Churn Prevention | CS, Product |
| 4 | Executive Report | Leadership |

**Time:** 3-4 hours

---

# SETUP: Load All Data

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
%matplotlib inline

# Load datasets
customers = pd.read_csv('../dataset/TechFlow.csv')
orders = pd.read_csv('../dataset/orders.csv')
products = pd.read_csv('../dataset/products.csv')
regions = pd.read_csv('../dataset/customer_regions.csv')
tickets = pd.read_csv('../dataset/support_tickets.tsv', sep='\t')
surveys = pd.read_csv('../dataset/nps_surveys.csv')
daily = pd.read_csv('../dataset/daily_metrics.csv')
daily['Date'] = pd.to_datetime(daily['Date'])

print('Datasets loaded!')
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


---
# PROJECT 1: Customer 360 Analysis

**Goal:** Create a complete view of each customer by combining all data sources.

**Deliverables:**
- Unified customer dataset
- Customer health scoring
- Risk categorization
- Visual dashboard

## 1.1 Build Customer Master Dataset

Start with base customer data and add ticket information.

```python
# Start with base data
customer_360 = customers.copy()

# Add ticket counts per customer
ticket_summary = tickets.groupby('CustomerID').agg(
    total_tickets=('TicketID', 'count'),
    high_priority=('Priority', lambda x: (x == 'High').sum())
).reset_index()

customer_360 = customer_360.merge(ticket_summary, on='CustomerID', how='left')
customer_360['total_tickets'] = customer_360['total_tickets'].fillna(0)

print(f'Shape: {customer_360.shape}')
customer_360[['CompanyName', 'total_tickets', 'high_priority']].head()
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 1.2 Add Survey Data

```python
# Aggregate NPS survey data
survey_summary = surveys.groupby('CustomerID').agg(
    survey_count=('SurveyID', 'count'),
    avg_survey_nps=('NPS_Score', 'mean'),
    latest_nps=('NPS_Score', 'last')
).reset_index()

customer_360 = customer_360.merge(survey_summary, on='CustomerID', how='left')
customer_360[['CompanyName', 'NPS_Score', 'avg_survey_nps', 'survey_count']].head(10)
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 1.3 Create Customer Segments

```python
# Revenue Tier
customer_360['RevenueTier'] = pd.cut(
    customer_360['MonthlyRevenue'],
    bins=[0, 100, 300, 600, 1000],
    labels=['Bronze', 'Silver', 'Gold', 'Platinum']
)

# Engagement Level
customer_360['EngagementLevel'] = pd.cut(
    customer_360['AvgWeeklyLogins'],
    bins=[0, 20, 50, 100, 500],
    labels=['Dormant', 'Low', 'Medium', 'High']
)

print('Revenue Tier Distribution:')
print(customer_360['RevenueTier'].value_counts())
print('\nEngagement Distribution:')
print(customer_360['EngagementLevel'].value_counts())
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 1.4 Calculate Health Score

```python
# Health Score (0-100)
# Components: NPS (30), Engagement (30), Tenure (20), Support (20)

customer_360['HealthScore'] = (
    (customer_360['NPS_Score'] / 10 * 30) +
    (customer_360['AvgWeeklyLogins'].clip(upper=100) / 100 * 30) +
    (customer_360['TenureMonths'].clip(upper=24) / 24 * 20) +
    (20 - customer_360['SupportTicketsRaised'].clip(upper=10) * 2)
).round(0)

# Risk Category
customer_360['RiskCategory'] = pd.cut(
    customer_360['HealthScore'],
    bins=[0, 40, 60, 80, 100],
    labels=['Critical', 'High', 'Medium', 'Low']
)

print('Health Score Statistics:')
print(customer_360['HealthScore'].describe())
print('\nRisk Distribution:')
print(customer_360['RiskCategory'].value_counts())
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 1.5 Analyze Risk by Segment

```python
# Risk summary by Revenue Tier
risk_by_tier = customer_360.groupby('RevenueTier').agg(
    customers=('CustomerID', 'count'),
    avg_health=('HealthScore', 'mean'),
    revenue_at_risk=('MonthlyRevenue', 'sum'),
    churn_rate=('Cancelled', 'mean')
).round(2)

risk_by_tier['churn_rate'] = (risk_by_tier['churn_rate'] * 100).round(1)
print('Risk Analysis by Revenue Tier:')
risk_by_tier
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 1.6 Customer 360 Dashboard

```python
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# 1. Revenue by Tier
tier_rev = customer_360.groupby('RevenueTier')['MonthlyRevenue'].sum()
axes[0,0].pie(tier_rev, labels=tier_rev.index, autopct='%1.1f%%')
axes[0,0].set_title('Revenue by Customer Tier')

# 2. Health Score Distribution
sns.histplot(customer_360['HealthScore'], bins=20, ax=axes[0,1], color='steelblue')
axes[0,1].axvline(60, color='orange', linestyle='--', label='Risk Threshold')
axes[0,1].set_title('Health Score Distribution')

# 3. Risk by Plan
risk_plan = pd.crosstab(customer_360['SubscriptionPlan'], customer_360['RiskCategory'], normalize='index') * 100
risk_plan.plot(kind='bar', stacked=True, ax=axes[0,2], colormap='RdYlGn_r')
axes[0,2].set_title('Risk by Plan (%)')

# 4. Engagement vs Revenue
colors = customer_360['RiskCategory'].map({'Critical':'red', 'High':'orange', 'Medium':'yellow', 'Low':'green'})
axes[1,0].scatter(customer_360['AvgWeeklyLogins'], customer_360['MonthlyRevenue'], c=colors, alpha=0.6)
axes[1,0].set_xlabel('Weekly Logins')
axes[1,0].set_ylabel('Revenue')
axes[1,0].set_title('Engagement vs Revenue')

# 5. Risk Distribution
risk_counts = customer_360['RiskCategory'].value_counts().reindex(['Low', 'Medium', 'High', 'Critical'])
axes[1,1].bar(risk_counts.index, risk_counts.values, color=['green', 'yellow', 'orange', 'red'])
axes[1,1].set_title('Customer Risk Distribution')

# 6. Industry Mix
industry_counts = customer_360['Industry'].value_counts().head(6)
axes[1,2].barh(industry_counts.index, industry_counts.values, color='purple')
axes[1,2].set_title('Top Industries')
axes[1,2].invert_yaxis()

plt.suptitle('CUSTOMER 360 DASHBOARD', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 1.7 Priority Action List

```python
# Top 10 at-risk customers by revenue
priority_list = customer_360[
    (customer_360['RiskCategory'].isin(['Critical', 'High'])) & 
    (customer_360['Cancelled'] == 0)
].nlargest(10, 'MonthlyRevenue')[[
    'CompanyName', 'Industry', 'MonthlyRevenue', 'HealthScore', 'NPS_Score', 'RiskCategory'
]]

print('='*70)
print('TOP 10 AT-RISK CUSTOMERS - IMMEDIATE ACTION REQUIRED')
print('='*70)
priority_list
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


---
# PROJECT 2: Revenue Intelligence

**Goal:** Deep analysis of revenue patterns and growth opportunities.

**Deliverables:**
- Revenue KPIs
- Concentration analysis
- Revenue at risk
- Growth opportunities

## 2.1 Revenue Overview

```python
total_mrr = customers['MonthlyRevenue'].sum()
total_arr = total_mrr * 12
avg_arpu = customers['MonthlyRevenue'].mean()

print('='*50)
print('REVENUE INTELLIGENCE REPORT')
print('='*50)
print(f'Total MRR: ${total_mrr:,}')
print(f'Total ARR: ${total_arr:,}')
print(f'Avg Revenue per Customer: ${avg_arpu:,.0f}')

# By plan
plan_revenue = customers.groupby('SubscriptionPlan').agg(
    customers=('CustomerID', 'count'),
    total_revenue=('MonthlyRevenue', 'sum'),
    avg_revenue=('MonthlyRevenue', 'mean')
).round(0)
plan_revenue['pct_of_total'] = (plan_revenue['total_revenue'] / total_mrr * 100).round(1)
print('\nRevenue by Plan:')
plan_revenue
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 2.2 Revenue Concentration (Pareto Analysis)

```python
# Sort customers by revenue
sorted_df = customers.sort_values('MonthlyRevenue', ascending=False).reset_index(drop=True)
sorted_df['CumulativeRevenue'] = sorted_df['MonthlyRevenue'].cumsum()
sorted_df['CumulativePct'] = sorted_df['CumulativeRevenue'] / total_mrr * 100

# Key metrics
top_10_pct = sorted_df.iloc[int(len(sorted_df)*0.1)]['CumulativePct']
top_20_pct = sorted_df.iloc[int(len(sorted_df)*0.2)]['CumulativePct']

print('REVENUE CONCENTRATION')
print('-'*40)
print(f'Top 10% of customers = {top_10_pct:.1f}% of revenue')
print(f'Top 20% of customers = {top_20_pct:.1f}% of revenue')
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 2.3 Pareto Chart

```python
fig, ax1 = plt.subplots(figsize=(12, 6))

ax1.bar(range(len(sorted_df)), sorted_df['MonthlyRevenue'], color='steelblue', alpha=0.7)
ax1.set_xlabel('Customers (sorted by revenue)')
ax1.set_ylabel('Monthly Revenue ($)', color='steelblue')

ax2 = ax1.twinx()
ax2.plot(range(len(sorted_df)), sorted_df['CumulativePct'], color='red', linewidth=2)
ax2.axhline(80, color='orange', linestyle='--', label='80% of revenue')
ax2.set_ylabel('Cumulative %', color='red')
ax2.legend()

plt.title('Revenue Concentration (Pareto Analysis)', fontsize=14, fontweight='bold')
plt.show()
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 2.4 Revenue at Risk

```python
# Already churned
churned_revenue = customers[customers['Cancelled']==1]['MonthlyRevenue'].sum()

# At-risk (using customer_360)
at_risk_revenue = customer_360[
    customer_360['RiskCategory'].isin(['Critical', 'High'])
]['MonthlyRevenue'].sum()

print('REVENUE AT RISK')
print('='*50)
print(f'Already Churned: ${churned_revenue:,}/month')
print(f'At-Risk (Critical+High): ${at_risk_revenue:,}/month')
print(f'\nAnnualized Impact: ${(churned_revenue + at_risk_revenue) * 12:,}/year')

# By industry
print('\nAt-Risk Revenue by Industry:')
at_risk = customer_360[customer_360['RiskCategory'].isin(['Critical', 'High'])]
print(at_risk.groupby('Industry')['MonthlyRevenue'].sum().sort_values(ascending=False).head(5))
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 2.5 Growth Opportunities

```python
# Upgrade candidates: Basic plan + High engagement + Good NPS
upgrade_candidates = customer_360[
    (customer_360['SubscriptionPlan'] == 'Basic') & 
    (customer_360['EngagementLevel'].isin(['Medium', 'High'])) &
    (customer_360['NPS_Score'] >= 7) &
    (customer_360['Cancelled'] == 0)
]

print('GROWTH OPPORTUNITIES')
print('='*50)
print(f'\n1. UPGRADE CANDIDATES')
print(f'   Count: {len(upgrade_candidates)} customers')
print(f'   Current Revenue: ${upgrade_candidates["MonthlyRevenue"].sum():,}')
print(f'   Potential (2x upgrade): ${upgrade_candidates["MonthlyRevenue"].sum() * 2:,}')

# Top industries
print(f'\n2. TOP PERFORMING INDUSTRIES')
industry_perf = customers.groupby('Industry').agg(
    customers=('CustomerID', 'count'),
    avg_revenue=('MonthlyRevenue', 'mean'),
    churn_rate=('Cancelled', 'mean')
).round(2)
industry_perf = industry_perf.sort_values('avg_revenue', ascending=False)
print(industry_perf.head(5))
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 2.6 Revenue Dashboard

```python
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Revenue by Plan (pie)
plan_rev = customers.groupby('SubscriptionPlan')['MonthlyRevenue'].sum()
axes[0,0].pie(plan_rev, labels=plan_rev.index, autopct='$%1.0f', colors=['#3498db','#2ecc71','#9b59b6'])
axes[0,0].set_title('Monthly Revenue by Plan')

# 2. Revenue by Industry
ind_rev = customers.groupby('Industry')['MonthlyRevenue'].sum().nlargest(6)
axes[0,1].barh(ind_rev.index, ind_rev.values, color='teal')
axes[0,1].set_title('Revenue by Industry')
axes[0,1].invert_yaxis()

# 3. Revenue distribution
sns.histplot(customers['MonthlyRevenue'], bins=20, ax=axes[1,0], color='steelblue')
axes[1,0].axvline(customers['MonthlyRevenue'].mean(), color='red', linestyle='--', label='Mean')
axes[1,0].set_title('Revenue Distribution')
axes[1,0].legend()

# 4. Revenue vs Seats
axes[1,1].scatter(customers['SeatCount'], customers['MonthlyRevenue'], alpha=0.6, c='purple')
axes[1,1].set_xlabel('Seat Count')
axes[1,1].set_ylabel('Monthly Revenue')
axes[1,1].set_title('Revenue vs Seats')

plt.suptitle('REVENUE INTELLIGENCE DASHBOARD', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


---
# PROJECT 3: Churn Prevention System

**Goal:** Build a comprehensive churn prediction and prevention framework.

**Deliverables:**
- Churn profile analysis
- Churn drivers identification
- Early warning system
- Priority intervention list

## 3.1 Churn Overview

```python
total = len(customers)
churned = customers['Cancelled'].sum()
churn_rate = churned / total * 100
revenue_lost = customers[customers['Cancelled']==1]['MonthlyRevenue'].sum()

print('CHURN OVERVIEW')
print('='*50)
print(f'Total Customers: {total}')
print(f'Churned: {churned} ({churn_rate:.1f}%)')
print(f'Monthly Revenue Lost: ${revenue_lost:,}')
print(f'Annual Revenue Lost: ${revenue_lost * 12:,}')
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 3.2 Churned vs Active Profile

```python
comparison = customers.groupby('Cancelled').agg({
    'MonthlyRevenue': 'mean',
    'TenureMonths': 'mean',
    'AvgWeeklyLogins': 'mean',
    'NPS_Score': 'mean',
    'SupportTicketsRaised': 'mean',
    'LastLoginDaysAgo': 'mean'
}).round(2).T

comparison.columns = ['Active', 'Churned']
comparison['Difference'] = comparison['Churned'] - comparison['Active']

print('CHURNED VS ACTIVE PROFILE')
print('='*50)
comparison
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 3.3 Churn by Segment

```python
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. By Plan
churn_plan = customers.groupby('SubscriptionPlan')['Cancelled'].mean() * 100
churn_plan.plot(kind='bar', ax=axes[0,0], color=['#3498db','#2ecc71','#9b59b6'])
axes[0,0].set_title('Churn Rate by Plan (%)')
axes[0,0].set_ylabel('Churn %')

# 2. By Contract
churn_contract = customers.groupby('ContractType')['Cancelled'].mean() * 100
churn_contract.plot(kind='bar', ax=axes[0,1], color=['#e74c3c','#27ae60'])
axes[0,1].set_title('Churn Rate by Contract (%)')

# 3. NPS by Churn
sns.boxplot(data=customers, x='Cancelled', y='NPS_Score', ax=axes[1,0], palette=['green','red'])
axes[1,0].set_xticklabels(['Active', 'Churned'])
axes[1,0].set_title('NPS: Active vs Churned')

# 4. Tenure by Churn
sns.boxplot(data=customers, x='Cancelled', y='TenureMonths', ax=axes[1,1], palette=['green','red'])
axes[1,1].set_xticklabels(['Active', 'Churned'])
axes[1,1].set_title('Tenure: Active vs Churned')

plt.suptitle('CHURN ANALYSIS BY SEGMENT', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 3.4 Churn Drivers (Correlation)

```python
numeric_cols = ['MonthlyRevenue', 'SeatCount', 'TenureMonths', 'AvgWeeklyLogins', 
                'NPS_Score', 'SupportTicketsRaised', 'LastLoginDaysAgo', 'Cancelled']

churn_corr = customers[numeric_cols].corr()['Cancelled'].drop('Cancelled').sort_values()

plt.figure(figsize=(10, 6))
colors = ['green' if x < 0 else 'red' for x in churn_corr.values]
plt.barh(churn_corr.index, churn_corr.values, color=colors)
plt.axvline(0, color='black', linewidth=0.5)
plt.xlabel('Correlation with Churn')
plt.title('CHURN DRIVERS', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print('\nPositive = Increases churn risk')
print('Negative = Decreases churn risk')
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 3.5 Early Warning System

```python
# Create warning flags
df = customer_360.copy()

df['Flag_LowNPS'] = df['NPS_Score'] < 6
df['Flag_LowEngagement'] = df['AvgWeeklyLogins'] < 15
df['Flag_HighTickets'] = df['SupportTicketsRaised'] > 4
df['Flag_NewCustomer'] = df['TenureMonths'] < 6
df['Flag_Inactive'] = df['LastLoginDaysAgo'] > 14
df['Flag_MonthlyContract'] = df['ContractType'] == 'Monthly'

# Count flags
flag_cols = [c for c in df.columns if c.startswith('Flag_')]
df['WarningCount'] = df[flag_cols].sum(axis=1)

# Risk level
df['ChurnRisk'] = pd.cut(df['WarningCount'], bins=[-1, 1, 3, 6], labels=['Low', 'Medium', 'High'])

print('EARLY WARNING INDICATORS')
print('='*50)
for flag in flag_cols:
    count = df[flag].sum()
    pct = count / len(df) * 100
    print(f'{flag.replace("Flag_", ""):20} {count:3} customers ({pct:.1f}%)')

print('\nRisk Distribution:')
print(df['ChurnRisk'].value_counts())
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 3.6 Priority Intervention List

```python
# High-value at-risk customers
priority = df[
    (df['ChurnRisk'] == 'High') & 
    (df['Cancelled'] == 0)
].nlargest(15, 'MonthlyRevenue')[[
    'CompanyName', 'Industry', 'MonthlyRevenue', 'NPS_Score', 
    'WarningCount', 'ChurnRisk'
]]

print('='*70)
print('PRIORITY INTERVENTION LIST')
print('High-value customers requiring immediate attention')
print('='*70)
priority
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


---
# PROJECT 4: Executive Strategy Report

**Goal:** Comprehensive business review with actionable recommendations.

**Deliverables:**
- Executive KPIs
- Department insights
- Full dashboard
- Strategic recommendations

## 4.1 Executive KPI Summary

```python
total_customers = len(customers)
active_customers = (customers['Cancelled']==0).sum()
churned = customers['Cancelled'].sum()
churn_rate = churned / total_customers * 100
mrr = customers['MonthlyRevenue'].sum()
arr = mrr * 12
avg_nps = customers['NPS_Score'].mean()

# NPS breakdown
promoters = (customers['NPS_Score'] >= 9).sum() / total_customers * 100
detractors = (customers['NPS_Score'] < 7).sum() / total_customers * 100
nps_score = promoters - detractors

print('='*60)
print('         TECHFLOW EXECUTIVE SUMMARY')
print('='*60)
print(f'\n📊 FINANCIAL')
print(f'   MRR: ${mrr:,}')
print(f'   ARR: ${arr:,}')
print(f'\n👥 CUSTOMERS')
print(f'   Total: {total_customers} | Active: {active_customers} | Churn: {churn_rate:.1f}%')
print(f'\n⭐ SATISFACTION')
print(f'   NPS: {nps_score:.0f} | Avg: {avg_nps:.1f}')
print(f'   Promoters: {promoters:.0f}% | Detractors: {detractors:.0f}%')
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 4.2 Department Insights

```python
print('DEPARTMENT INSIGHTS')
print('='*60)

# SALES
print('\n📈 SALES')
plan_mix = customers.groupby('SubscriptionPlan')['CustomerID'].count()
print(f'   Plan Mix: Basic={plan_mix.get("Basic",0)}, Standard={plan_mix.get("Standard",0)}, Enterprise={plan_mix.get("Enterprise",0)}')
print(f'   Top Industry: {customers.groupby("Industry")["MonthlyRevenue"].sum().idxmax()}')

# CUSTOMER SUCCESS
print('\n🎯 CUSTOMER SUCCESS')
at_risk = len(df[df['ChurnRisk']=='High'])
print(f'   At-Risk Customers: {at_risk}')
print(f'   Avg Support Tickets: {customers["SupportTicketsRaised"].mean():.1f}')

# PRODUCT
print('\n💻 PRODUCT')
print(f'   Avg Weekly Logins: {customers["AvgWeeklyLogins"].mean():.0f}')
high_engage = (customers['AvgWeeklyLogins'] > 50).mean() * 100
print(f'   High Engagement: {high_engage:.0f}%')

# FINANCE
print('\n💰 FINANCE')
churned_rev = customers[customers['Cancelled']==1]['MonthlyRevenue'].sum()
print(f'   Revenue Churn: ${churned_rev:,}/month')
annual_pct = len(customers[customers['ContractType']=='Annual']) / total_customers * 100
print(f'   Annual Contracts: {annual_pct:.0f}%')
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 4.3 Executive Dashboard

```python
fig = plt.figure(figsize=(18, 12))

# KPI Cards
kpis = [('MRR', f'${mrr:,}', '#3498db'), ('Customers', str(total_customers), '#2ecc71'),
        ('Churn', f'{churn_rate:.1f}%', '#e74c3c'), ('NPS', f'{nps_score:.0f}', '#9b59b6')]

for i, (label, value, color) in enumerate(kpis):
    ax = fig.add_subplot(4, 4, i+1)
    ax.text(0.5, 0.55, value, fontsize=28, fontweight='bold', ha='center', color=color)
    ax.text(0.5, 0.2, label, fontsize=11, ha='center')
    ax.axis('off')

# Revenue by Plan
ax1 = fig.add_subplot(4, 3, 4)
plan_rev = customers.groupby('SubscriptionPlan')['MonthlyRevenue'].sum()
ax1.pie(plan_rev, labels=plan_rev.index, autopct='%1.0f%%')
ax1.set_title('Revenue by Plan')

# Risk Distribution
ax2 = fig.add_subplot(4, 3, 5)
risk_counts = df['ChurnRisk'].value_counts().reindex(['Low', 'Medium', 'High'])
ax2.bar(risk_counts.index, risk_counts.values, color=['green','orange','red'])
ax2.set_title('Customer Risk')

# Top Industries
ax3 = fig.add_subplot(4, 3, 6)
top_ind = customers.groupby('Industry')['MonthlyRevenue'].sum().nlargest(5)
ax3.barh(top_ind.index, top_ind.values, color='steelblue')
ax3.set_title('Top Industries')
ax3.invert_yaxis()

# Tenure Distribution
ax4 = fig.add_subplot(4, 3, 7)
sns.histplot(customers['TenureMonths'], bins=15, ax=ax4, color='teal')
ax4.set_title('Tenure Distribution')

# NPS Distribution
ax5 = fig.add_subplot(4, 3, 8)
sns.histplot(customers['NPS_Score'], bins=10, ax=ax5, color='purple')
ax5.axvline(7, color='orange', linestyle='--')
ax5.set_title('NPS Distribution')

# Churn by Plan
ax6 = fig.add_subplot(4, 3, 9)
churn_seg = customers.groupby('SubscriptionPlan')['Cancelled'].mean() * 100
ax6.bar(churn_seg.index, churn_seg.values, color=['#3498db','#2ecc71','#9b59b6'])
ax6.set_title('Churn by Plan (%)')

# Engagement vs Revenue
ax7 = fig.add_subplot(4, 3, 10)
ax7.scatter(customers['AvgWeeklyLogins'], customers['MonthlyRevenue'], 
            c=customers['NPS_Score'], cmap='RdYlGn', alpha=0.6)
ax7.set_xlabel('Logins')
ax7.set_ylabel('Revenue')
ax7.set_title('Engagement vs Revenue')

# Correlation
ax8 = fig.add_subplot(4, 3, 11)
corr_cols = ['MonthlyRevenue', 'NPS_Score', 'AvgWeeklyLogins', 'Cancelled']
sns.heatmap(customers[corr_cols].corr(), annot=True, fmt='.2f', cmap='RdYlGn', ax=ax8)
ax8.set_title('Correlations')

# Contract Mix
ax9 = fig.add_subplot(4, 3, 12)
contract = customers['ContractType'].value_counts()
ax9.pie(contract, labels=contract.index, autopct='%1.0f%%', colors=['#e74c3c','#27ae60'])
ax9.set_title('Contract Mix')

plt.suptitle('TECHFLOW EXECUTIVE DASHBOARD', fontsize=20, fontweight='bold')
plt.tight_layout()
plt.show()
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


## 4.4 Strategic Recommendations

```python
high_risk_count = len(df[df['ChurnRisk']=='High'])
high_risk_rev = df[df['ChurnRisk']=='High']['MonthlyRevenue'].sum()
basic_count = len(upgrade_candidates)
monthly_count = len(customers[customers['ContractType']=='Monthly'])

print('='*70)
print('            STRATEGIC RECOMMENDATIONS')
print('='*70)

print(f'''\n🎯 IMMEDIATE PRIORITIES (30 Days)\n{'-'*50}\n1. SAVE AT-RISK REVENUE\n   - {high_risk_count} high-risk customers\n   - ${high_risk_rev:,}/month at stake\n   - Action: Dedicated CSM outreach\n\n2. ADDRESS DETRACTORS\n   - {int(detractors/100*total_customers)} NPS detractors\n   - Action: Personal outreach within 48 hours''')

print(f'''\n\n📈 GROWTH (Next Quarter)\n{'-'*50}\n1. UPSELL CAMPAIGN\n   - {basic_count} Basic customers ready for upgrade\n   - Target: High engagement + Good NPS\n\n2. ANNUAL CONTRACT PUSH\n   - {monthly_count} monthly customers\n   - Offer: 2 months free for annual''')

print(f'''\n\n⚠️ RISK MITIGATION\n{'-'*50}\n1. New customer onboarding improvement\n2. Weekly engagement monitoring\n3. Proactive support for high-ticket accounts''')

print('\n' + '='*70)
```

In [ ]:
# ↓ Type the code below, then press Shift+Enter to run


---
# 🎉 CONGRATULATIONS!

You have completed the TechFlow Data Analysis Training Series!

**Skills Mastered:**
- ✅ Python Fundamentals (Modules 0.1-0.9)
- ✅ Pandas DataFrames & Series (Module 1)
- ✅ Filtering, Grouping, Aggregation (Module 2)
- ✅ Joining Multiple Datasets (Module 3)
- ✅ Data Cleaning & Exploration (Module 4)
- ✅ Data Visualization (Module 5)
- ✅ Real-World Business Analysis (Module 6)

**You are now ready to analyze data like a pro!** 🚀